% Created by: Azizbek Asadov %
% Date: 02.10.2024

Equity data - D&J 60 from 2019-11-01 to 2020-11-01 (source: finance yahoo)

In [ ]:
import yfinance as yf
import pandas as pd

tickers = [
    'MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 
    'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 
    'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'VZ', 'V', 
    'WBA', 'WMT', 'RTX', 'TSLA', 'META', 'GOOGL', 'GOOG', 'NFLX', 'AMZN', 'BMY', 'PEP', 
    'ORCL', 'QCOM', 'TXN', 'ADP', 'COST', 'GE', 'MO', 'ABT', 'T', 
    'SBUX', 'LLY', 'BRK-B', 'PFE', 'XOM', 'MDT', 'TGT', 'SPGI', 
    'BK', 'C', 'SLB', 'DHR', 'DUK'
]
start_date = '2019-11-01'
end_date = '2020-11-01'

data = yf.download(tickers, start=start_date, end=end_date, interval='1d')['Close'])
data.to_csv('DJ60_securities.csv')


Risk-free data - T-bill 10Y yield for the same time period (source: finance yahoo)

In [ ]:
import yfinance as yf
import pandas as pd

ticker = "^TNX"

y10_data = yf.download(ticker, start=start_date, end=end_date, interval='1d')['Close']y10_data.head()

y10_data.to_csv('10Y_Treasury_Yield.csv')
rf_rate = y10_data['Close']/100descriptive_stats = rf_rate.describe()
descriptive_stats.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(y10_data['Close'], label='10Y Treasury Yield')
plt.title('10-Year Treasury Yield (2019-11-01 to 2020-11-01)')
plt.xlabel('Date')
plt.ylabel('Yield')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compute daily returns
daily_returns = data.pct_change().dropna()[:20]

# Compute log returns
log_returns = np.log(data / data.shift(1))
# Compound returns
compound_returns = (1 + daily_returns).prod() - 1

# Descriptive statistics (First 4 moments)
mean_returns = log_returns.mean()
variance = log_returns.var()
skewness = log_returns.skew()
kurtosis = log_returns.kurtosis()

# Combine the descriptive statistics into a DataFrame
descriptive_stats = pd.DataFrame({
    'Mean': mean_returns,
    'Variance': variance,
    'Skewness': skewness,
    'Kurtosis': kurtosis
})

# Calculate the correlation matrix
correlation_matrix = log_returns.corr()

# Display the compound returns and descriptive statistics
print("Compound Returns:\n", compound_returns)
print("\nDescriptive Statistics (First 4 moments):\n", descriptive_stats)

selected_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TSLA']
# Plot the correlation matrix as a heatmap
for ticker in selected_tickers:
    plt.plot(data.index, data[ticker], label=ticker)

plt.figure(figsize=(12, 8))
plt.title('Correlation Matrix')
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.show()